In [1]:
%pwd

'c:\\Users\\44787\\Desktop\\projects\\End-To-End-MLOps-Project\\notebooks'

In [2]:
import os

os.chdir("../")
%pwd

'c:\\Users\\44787\\Desktop\\projects\\End-To-End-MLOps-Project'

In [3]:
from src.utils.common import read_yaml
from src.constants import PARAMS_FILE_PATH

import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import joblib


import warnings
warnings.filterwarnings("ignore")

In [49]:
config = read_yaml(PARAMS_FILE_PATH)

remote_server_uri = config.mlflow_config.remote_server_uri
# mlflow.set_tracking_uri(remote_server_uri)
mlflow.set_tracking_uri("http://localhost:1234")



# runs = mlflow.search_runs(experiment_ids=[0,1,2])
runs = mlflow.search_runs(experiment_ids=[3])

runs


[2024-05-26 21:52:14,190: INFO: common: yaml file: params.yaml loaded successfully]


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.acu,metrics.f1_score,params.min_samples_split,params.min_samples_leaf,params.max_depth,params.criterion,params.n_estimators,tags.mlflow.runName,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.source.git.commit,tags.mlflow.log-model.history,tags.mlflow.source.name
0,02f0dd28aae04c77bb200015f4599910,3,FINISHED,file:///C:/Users/44787/Desktop/projects/End-To...,2024-05-26 20:50:52.081000+00:00,2024-05-26 20:51:15.147000+00:00,0.985082,0.985257,2,1,30,entropy,10,Random_Forest,44787,LOCAL,6716002dac22a89e10efba2403cdf40fccccb06b,"[{""run_id"": ""02f0dd28aae04c77bb200015f4599910""...",.\src\pipeline\stage_03_training_and_evaluatio...
1,41a6f3ea85f04cec8a893a2d06976582,3,FINISHED,file:///C:/Users/44787/Desktop/projects/End-To...,2024-05-26 20:49:48.788000+00:00,2024-05-26 20:50:09.559000+00:00,0.977881,0.978294,4,2,30,gini,200,Random_Forest,44787,LOCAL,6716002dac22a89e10efba2403cdf40fccccb06b,"[{""run_id"": ""41a6f3ea85f04cec8a893a2d06976582""...",.\src\pipeline\stage_03_training_and_evaluatio...


In [50]:
runs["metrics.acu"].sort_values(ascending=False, ignore_index=True)

0    0.985082
1    0.977881
Name: metrics.acu, dtype: float64

In [51]:
highest = runs["metrics.acu"].sort_values(ascending=False, ignore_index=True)[0]
highest

0.9850823045267489

In [52]:
runs[runs["metrics.acu"]==highest]

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.acu,metrics.f1_score,params.min_samples_split,params.min_samples_leaf,params.max_depth,params.criterion,params.n_estimators,tags.mlflow.runName,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.source.git.commit,tags.mlflow.log-model.history,tags.mlflow.source.name
0,02f0dd28aae04c77bb200015f4599910,3,FINISHED,file:///C:/Users/44787/Desktop/projects/End-To...,2024-05-26 20:50:52.081000+00:00,2024-05-26 20:51:15.147000+00:00,0.985082,0.985257,2,1,30,entropy,10,Random_Forest,44787,LOCAL,6716002dac22a89e10efba2403cdf40fccccb06b,"[{""run_id"": ""02f0dd28aae04c77bb200015f4599910""...",.\src\pipeline\stage_03_training_and_evaluatio...


In [53]:
runs[runs["metrics.acu"]==highest]['run_id']

0    02f0dd28aae04c77bb200015f4599910
Name: run_id, dtype: object

In [54]:
highest_run_id = runs[runs["metrics.acu"]==highest]["run_id"]
highest_run_id = highest_run_id.reset_index(drop=True)[0]
highest_run_id

'02f0dd28aae04c77bb200015f4599910'

In [55]:
model_name = config.mlflow_config.registered_model_name

client = MlflowClient()
for mv in client.search_model_versions(f"name='{model_name}'"):
    mv = dict(mv)
    # print(mv)

    if mv["run_id"] == highest_run_id:
        current_version = mv["version"]
        logged_model = mv["source"]
        pprint(mv, indent=4)

{   'aliases': [],
    'creation_timestamp': 1716756675133,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1716756675133,
    'name': 'RF_Register',
    'run_id': '02f0dd28aae04c77bb200015f4599910',
    'run_link': '',
    'source': 'file:///C:/Users/44787/Desktop/projects/End-To-End-MLOps-Project/artifacts/3/02f0dd28aae04c77bb200015f4599910/artifacts/models_mlflow',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '2'}


In [56]:
model_name = config.mlflow_config.registered_model_name

client = MlflowClient()
for mv in client.search_model_versions(f"name='{model_name}'"):
    mv = dict(mv)

    if mv["run_id"] == highest_run_id:
        current_version = mv["version"]
        logged_model = mv["source"]
        pprint(mv, indent=4)
        client.transition_model_version_stage(
            name=model_name,
            version=current_version,
            stage="Production"
        )
    else:
        current_version = mv["version"]
        client.transition_model_version_stage(
            name=model_name,
            version=current_version,
            stage="Staging"
        )    

{   'aliases': [],
    'creation_timestamp': 1716756675133,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1716756675133,
    'name': 'RF_Register',
    'run_id': '02f0dd28aae04c77bb200015f4599910',
    'run_link': '',
    'source': 'file:///C:/Users/44787/Desktop/projects/End-To-End-MLOps-Project/artifacts/3/02f0dd28aae04c77bb200015f4599910/artifacts/models_mlflow',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '2'}


In [57]:
loaded_model = mlflow.pyfunc.load_model(logged_model)

model_path = config.webapp_model_dir
joblib.dump(loaded_model, model_path)

['saved_models/model.joblib']